# Les bases de la syntaxe Keras

Avec TensorFlow 2.x , Keras est désormais le principal choix d'API. Nous allons travailler sur un projet de régression simple pour comprendre les bases de la syntaxe de Keras et de l'ajout de couches.

## Les données

Pour apprendre la syntaxe de base de Keras, nous utiliserons un ensemble de fausses données très simple. Dans les sessions suivantes, nous nous concentrerons sur des ensembles de données réels, ainsi que sur l'ingénierie des caractéristiques (feature engineering) ! Pour l'instant, concentrons-nous sur la syntaxe de TensorFlow 2.x.

Imaginons que ces données soient les mesures de quelques pierres précieuses rares, avec deux caractéristiques de mesure et un prix de vente. Notre objectif final serait d'essayer de prédire le prix de vente d'une nouvelle pierre précieuse que nous venons d'extraire du sol, afin d'essayer de fixer un prix équitable sur le marché.

### Chargement des données

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('fake_reg.csv')

In [ ]:
df.head()

## Exploration des données

Jetons un coup d'œil rapide, nous devrions constater une forte corrélation entre les caractéristiques (features) et le "prix" de ce produit composé.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.pairplot(df);

## Répartition Entraînement / Test (Train / Test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

# Features 
X = df[['feature1','feature2']].values

# Label 
y = df['price'].values

# Split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

## Normalisation et mise à l'échelle des données

Nous mettons à l'échelle les données relatives aux caractéristiques.

[Pourquoi nous n'avons pas besoin de mettre le label à l'échelle] 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
help(MinMaxScaler)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Syntaxe TensorFlow 2.x


## Options d'importation

Vous pouvez importer Keras de Tensorflow de plusieurs façons (c'est un choix très personnel, veuillez utiliser les méthodes d'importation que vous préférez). Nous utiliserons la méthode indiquée dans la **documentation officielle de TF**.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
help(Sequential)

## Création d'un modèle

Il y a deux façons de créer des modèles via l'API TF 2 Keras, soit en passant une liste de couches en une seule fois, soit en les ajoutant une par une.

Montrons les deux méthodes.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

### Modèle - avec liste de couches

In [ ]:
model = Sequential([
    Dense(units=2),
    Dense(units=2),
    Dense(units=2)
])

### Modèle - ajout des couches une par une

In [ ]:
model = Sequential()

model.add(Dense(2))
model.add(Dense(2))
model.add(Dense(2))

Continuons et construisons un modèle simple, puis compilons-le en définissant notre solveur

In [ ]:
model = Sequential()

model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))

# Couche finale pour notre prédiction avec un seul noeud de sortie
model.add(Dense(1))

model.compile(optimizer='rmsprop',loss='mse')

### Choix d'un optimiseur (optimizer) et perte (loss)

Gardez à l'esprit le type de problème que vous essayez de résoudre :

    # Pour un problème de classification multi-classes
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Pour un problème de classification binaire
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Pour un problème de régression de l'erreur quadratique moyenne
    model.compile(optimizer='rmsprop',
                  loss='mse')

## Entraînement

Vous trouverez ci-dessous quelques définitions courantes qu'il est nécessaire de connaître et de comprendre pour utiliser correctement Keras :

* Sample (Échantillon) : un élément d'un ensemble de données.
    * Exemple : une image est un échantillon dans un réseau convolutif
    * Exemple : un fichier audio est un échantillon pour un modèle de reconnaissance vocale
* Batch (Lot) : un ensemble de N échantillons. Les échantillons d'un batch sont traités indépendamment, en parallèle. En cas d'entraînement, un batch ne donne lieu qu'à une seule mise à jour du modèle. Un batch se rapproche généralement davantage de la distribution des données d'entrée qu'une seule entrée. Plus le batch est important, meilleure est l'approximation ; cependant, il est également vrai que le traitement du batch sera plus long et qu'il ne donnera lieu qu'à une seule mise à jour. Pour l'inférence (évaluation/prédiction), il est recommandé de choisir un batch aussi grand que possible sans épuiser la mémoire (car des batchs plus grands entraînent généralement une évaluation/prédiction plus rapide).
* Epoch (période d'entraînement) : un seuil arbitraire, généralement défini comme "un passage sur l'ensemble des données", utilisée pour séparer l'entraînement en phases distinctes, ce qui est utile pour l'enregistrement et l'évaluation périodique.
* En utilisant validation_data ou validation_split avec la méthode d'ajustement des modèles de Keras, l'évaluation sera effectuée à la fin de chaque Epoque.
* Dans Keras, il est possible d'ajouter des rappels spécifiquement conçus pour être exécutés à la fin d'une Epoque. Il s'agit par exemple de changements de taux d'apprentissage et de contrôle (sauvegarde) des modèles.

In [ ]:
model.fit(X_train,y_train,epochs=250)

## Évaluation

Évaluons nos performances sur notre ensemble d'entraînement et sur notre ensemble de test. Nous pouvons comparer ces deux performances pour vérifier qu'il n'y a pas d'overfitting (ou sur-apprentissage).

In [ ]:
model.history.history

In [ ]:
loss = model.history.history['loss']

In [ ]:
sns.lineplot(x=range(len(loss)),y=loss)
plt.title("Perte sur le set d'entraînement par Epoch");

### Comparation de l'évaluation finale sur le set d'entraînement et le set de test.

Il faut espérer qu'ils soient assez proches l'un de l'autre.

In [ ]:
model.metrics_names

In [ ]:
training_score = model.evaluate(X_train,y_train,verbose=0)
test_score = model.evaluate(X_test,y_test,verbose=0)

In [ ]:
training_score

In [ ]:
test_score

### Évaluations complémentaires

In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
test_predictions

In [ ]:
pred_df = pd.DataFrame(y_test,columns=['Test Y'])

In [ ]:
pred_df

In [ ]:
test_predictions = pd.Series(test_predictions.reshape(300,))

In [ ]:
test_predictions

In [ ]:
pred_df = pd.concat([pred_df,test_predictions],axis=1)

In [ ]:
pred_df.columns = ['Test Y','Model Predictions']

In [ ]:
pred_df

Comparons avec les vrais labels de test !

In [ ]:
sns.scatterplot(x='Test Y',y='Model Predictions',data=pred_df)

In [ ]:
pred_df['Error'] = pred_df['Test Y'] - pred_df['Model Predictions']

In [ ]:
sns.displot(pred_df['Error'],bins=50)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
mean_absolute_error(pred_df['Test Y'],pred_df['Model Predictions'])

In [ ]:
mean_squared_error(pred_df['Test Y'],pred_df['Model Predictions'])

In [ ]:
# Essentiellement la même chose, la différence est due à la précision
test_score

In [ ]:
#RMSE
test_score**0.5

## Prédire sur de toutes nouvelles données


In [ ]:
# [[Feature1, Feature2]]
new_gem = [[998,1000]]

In [ ]:
# N'oubliez pas de mettre à l'échelle !
scaler.transform(new_gem)

In [ ]:
new_gem = scaler.transform(new_gem)

In [ ]:
model.predict(new_gem)

## Sauvegarde et chargement d'un modèle

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('my_model.h5') # crée un fichier HDF5 'mon_modèle.h5

In [ ]:
later_model = load_model('my_model.h5')

In [ ]:
later_model.predict(new_gem)